# Data Collection

## I. Sources

We will collect county level demographic data and presidential election results from the four following election years:

In [55]:
years = [2008, 2012, 2016, 2020]

Presidential election data is obtained from the <a href="https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ">MIT Election Data + Science Lab</a>.

>> MIT Election Data and Science Lab, 2018, "County Presidential Election Returns 2000-2020", https://doi.org/10.7910/DVN/VOQCHQ, Harvard Dataverse, V12, UNF:6:KNR0/XNVzJC+RnAqIx5Z1Q== [fileUNF] 


Demographic data is obtained from the IPUMS's National Historical Geographic Information System (NHGIS). Specifically, we use their data coming from the US American Community Survey 5-year estimates. For election year $n$, we use the survey that covers the range $n-2$ to $n+2$. The variable names and their meanings can be found in the codebooks that accompony the NHGIS datasets. Throughout the cleaning process, we replace the original variable codes with mnemonically appropriate names. 

>>Steven Manson, Jonathan Schroeder, David Van Riper, Katherine Knowles, Tracy Kugler, Finn Roberts, and Steven Ruggles. IPUMS National Historical Geographic Information System: Version 18.0 [dataset]. Minneapolis, MN: IPUMS. 2023. http://doi.org/10.18128/D050.V18.0


Geographic data for all counties is taken directly from the pygris package.

## II. Importing data

In [56]:
import pandas as pd
import numpy as np
import pygris as pyg
import area_unit_conversion as auc

Begin by importing all data, creating dictionaries for each type of data, and splitting data according to years.

In [57]:
# Import election data
raw_election_data = pd.read_csv("raw_data/MIT_election_data/president_2000-2020_by_county.csv")

# Create dictionaries to hold data by year
election = {}
demographic = {}
geographic = {}
data = {}

#  Add to dictionaries with { key=year : value =  data for that year}
# e.g. demographic[2008] gives access to the 2008 demographic data
# e.g. election[2008] gives access to the 2008 election data
for year in years:
    # Add election data
    election[year] = raw_election_data[ raw_election_data.year == year ]
    # Add demographic data
    demographic[year] = pd.read_csv(f"raw_data/NHGIS/general/general_{year - 2}_{year + 2}.csv",encoding = 'unicode_escape')
    # Add geographic data
    if year<2014:  
        geographic[year] = pyg.counties(cb=True,cache=True, year=year+2)
    else:
        geographic[year] = pyg.counties(cb=True,cache=True, year=year)

# Data Handling Functions

## I. Auxiliary Functions

### Find missing values

In [58]:
# Returns a dictionary with (keys= column names , values= list of row indices in that column with NaN)
def find_nan(df):
    return { feature : df[feature][df[feature].isna()].index.to_list() for feature in df.columns.to_list() }

### Find zeros

In [59]:
# Returns a list of row indices which have zeros in the specified column
def find_zeros(df, column):
    return df[ df[column]==0 ].index.to_list()

### Find NHGIS codes for race-demographic features

In [60]:
#Returns a list of race-demographics related variables depending on the input year.
def race_dict(year):
    if (year == 2020):
        return ['AQYHE','AQYIE','AQYJE','AQYKE','AQYLE','AQYME','AQYNE']
    if (year == 2016):
        return ['AJ6OE','AJ6PE','AJ6QE','AJ6RE','AJ6SE','AJ6TE','AJ6UE']
    if (year == 2012):
        return ['ABK1E','ABK2E','ABK3E','ABK4E','ABK5E','ABK6E','ABK7E']
    if (year == 2008):
        return ['JVOE','JVPE','JVQE','JVRE','JVSE','JVTE','JVUE'] 

### Convert from square-meters to square-miles

In [61]:
def f(x):
    return round(auc.convert('m2','mi2',x),3)

### Find mismatched FIPS codes between a list different data sets

In [62]:
def fips_dropper(df_list):
    #Make list of fips codes in each dataframe in df_list
    fip_list = [ set(df['fips'].values) for df in df_list ]

    #Compute the intersection of all the lists of fips codes
    fip_common = fip_list[0]
    for i in range(1,len(fip_list)):
        fip_common = fip_common.intersection(fip_list[i])
        
    #Drop the non-common fips rows and sort by fips
    for df in df_list:
        rows_to_drop = [ j for j in range(len(df)) if df.loc[j,'fips'] not in fip_common ]
        df.drop(index=rows_to_drop, inplace=True)
        df.sort_values(by='fips',inplace=True,ignore_index=True)
    return df_list

## II. Cleaning and merging functions

### Election Data Cleaner

In [63]:
def election_cleaner():
    # Creates a list with all county FIP codes
    list_of_fips = pd.read_csv("../data/RANDOM/fips2county.tsv", sep="\t")
    list_of_fips = list_of_fips[['CountyFIPS']]
    
    # Goes through each election year
    for year in election:
        # Resets index of each year from zero
        election[year].reset_index(inplace=True,drop=True)
        
        # Rename columns
        election[year] = election[year].rename({'county_name':'county', 'county_fips':'fips', 'candidatevotes':'votes'}, axis='columns')
        
        # Creates a list of rows to drop
        rows_to_drop = []
        
        # Find rows with missing FIP codes
        missing_fips = find_nan(election[year])['fips']
        rows_to_drop += missing_fips
        
        # Find rows with missing votes (totalvotes=0)
        missing_votes = find_zeros(election[year], 'totalvotes')
        rows_to_drop += missing_votes
        
        # Find rows corresponding to third parties
        third_parties = election[year][ election[year]['party'] != 'DEMOCRAT'][ election[year]['party'] != 'REPUBLICAN'].index.to_list()
        rows_to_drop += third_parties
        
        # Find rows with incorrect/nonexistent FIP codes
        weird_fips = list(set(election[year]['fips'])-set(list_of_fips['CountyFIPS']))
        weird_fips_rows = []
        for i in range(len(election[year])):
            if election[year].loc[i,'fips'] in weird_fips:
                weird_fips_rows.append(i)
        rows_to_drop += weird_fips_rows
        
        # Drop problematic rows and resets index
        rows_to_drop = list(set(rows_to_drop))
        election[year] = election[year].drop(rows_to_drop, axis=0)
        election[year].reset_index(inplace=True,drop=True)
        
        # Deal with 2020 election year, which requires combining different modes of voting
        if year==2020:
            df_temp = pd.DataFrame(columns=election[year].columns)  # new temp df
            for row in range(len(election[year])):                  # go through each row of 2020 election
                if ( election[year].loc[row,'mode'] == 'TOTAL'):    # if the mode is total, add row to temp df
                    df_temp.loc[len(df_temp)] = election[year].loc[row].values
                elif (election[year].loc[row, 'mode']=='SKIP'):     # if the mode is skip, skip that row
                    continue
                else:                                               # deal with non-total rows
                    j = 0                                           # set a counter
                    total_votes = 0                                 # new variable to keep track of cumulative sum
                    while (election[year].loc[row+j,'fips']==election[year].loc[row,'fips'] ) and (election[year].loc[row+j,'party']==election[year].loc[row,'party']):
                        total_votes += election[year].loc[row+j,'votes']
                        if row+j> row:
                            election[year].loc[row+j,'mode'] = 'SKIP'
                        j = j + 1
                    df_temp.loc[len(df_temp)] = election[year].loc[row].values
                    df_temp.loc[len(df_temp)-1,'mode'] = 'TOTAL'
                    df_temp.loc[len(df_temp)-1,'votes'] = total_votes
            election[year] = df_temp
        
        # Calculate and add columns containing dem, repub, other votes
        other_votes = [election[year].loc[2*i,'totalvotes']-election[year].loc[2*i,'votes']-election[year].loc[2*i+1,'votes'] for i in range(int(len(election[year])/2))]
        repub_votes = [ election[year].loc[2*i+1,'votes'] for i in range(int(len(election[year])/2))]
        repub = [2*i+1 for i in range(int(len(election[year])/2))]  # find republican rows (odd rows)
        election[year] = election[year].drop(repub, axis=0)         # drop republican rows
        election[year].reset_index(inplace=True,drop=True)          # reset index
        election[year]['dem'] = election[year].votes                # add dem votes column
        election[year]['repub'] = repub_votes                       # add repub votes column
        election[year]['other'] = other_votes                       # add other votes column

        # Cast FIPs as integers
        election[year].fips = election[year].fips.astype(int)

        # Drop unnecessary columns
        cols_to_drop = ['year', 'state', 'state_po', 'county', 'office', 'candidate', 'votes', 'party', 'version']
        election[year] = election[year].drop(cols_to_drop, axis=1)
    
    # Do a final row elimination, to keep only the rows that all election years have in common
    final_fips_to_drop = list(set(election[2020].fips)-set.intersection(set(election[2008].fips),set(election[2012].fips),set(election[2016].fips),set(election[2020].fips)))
    rows_to_drop = []
    for year in election:
        for fips in final_fips_to_drop:
            if fips in election[year].fips.values:
                election[year] = election[year].drop(election[year][ election[year].fips==fips ].index.to_list(), axis=0)
                election[year].reset_index(inplace=True,drop=True)

### Demographic data cleaner

In [64]:
def demo_cleaner():
    for year in years:
        df = demographic[year]

        #Drop all entries corresponding to Puerto Rico
        df.drop(index = df.index[df['STATE'] == "Puerto Rico"].tolist(), inplace=True)
        df.reset_index(inplace=True,drop=True)

        #Fix the erroneous fips code for Connecticut (state code = 9) for the year 2020
        if (year==2020):
            for i in df.index[df['STATEFP'] == 9].tolist():
                df.loc[i,'COUNTYFP'] //= 10

        #Create the output dataframe; we do this here so we can get rid of columns in df
        new_df = pd.DataFrame()
        new_df['fips'] = 1000* df['STATEFP'] + df['COUNTYFP']

        #drop irrelevant columns (first 9) and margin-of-error columns in df
        cols = list(df.columns)
        cols_to_drop = cols[:9] + [c for c in cols if c[-1]=='M']
        df.drop(inplace=True,columns=cols_to_drop)

        #Remove last 3 digits from each column name in df; 
        #This is the only thing that varies by year.
        cols = list(df.columns)
        col_map = { cols[i] : (cols[i])[:-3] for i in range(len(cols)) }
        df.rename(col_map, inplace = True, axis = 'columns')

        #Rename and add some of the columns to the new df
        col_names = [['AV0AA','pop_tot'],
                    ['AV1AA','pop_male'],
                    ['AV1AB','pop_female'],
                    ['A35AA','hispanic'],
                    ['B84AA','labor_total'],
                    ['B84AB','labor_armed'],
                    ['B84AD','labor_employed'],
                    ['B84AE','labor_unemployed'],
                    ['B79AA','income_median'],
                    ['BD5AA','income_percapita'],
                    ['CL6AA','income_poverty'],
                    ['AT5AA','native_yes'],
                    ['AT5AB','native_no'],
                    ['AR5AA','houses_tot'],
                    ['BS7AA','income_10'],
                    ['BS7AB','income_10-15'],
                    ['BS7AC','income_15-25'],
                    ['BS7AD','income_25']
                    ]

        for i in range(len(col_names)):
            new_df[col_names[i][1]] = df[col_names[i][0]]

        #Add columns on marital status to new df
        new_df['marital_single']  = df['BL1AA'] + df['BL1AG']
        new_df['marital_married'] = df['BL1AC'] + df['BL1AI']
        new_df['marital_sepdiv']  = df['BL1AD'] + df['BL1AF'] + df['BL1AJ'] + df['BL1AL']
        new_df['marital_widow']   = df['BL1AE'] + df['BL1AK']

        #Add columns on educational attainment
        new_df['edu_low']  = df['B87AA'] + df['B87AB']
        new_df['edu_mid']  = df['B87AC'] + df['B87AD'] + df['B87AE']
        new_df['edu_high'] = df['B87AF'] + df['B87AG']

        #Add columns on edu attainment by age
        cols = list(df.columns)
        edu = [['edu_low',0,1],
            ['edu_mid',2,3,4],
            ['edu_high',5,6]]
        age = [['age_18',0,7,14],
            ['age_45',21],
            ['age_65',28]]
        ind = cols.index('B91AA')

        for i in range(3):
            for j in range(3):
                e = edu[i]
                a = age[j]
                name = e[0] + '_' + a[0]
                new_df[name] = 0
                for l in a[1:]:
                        for m in e[1:]:
                            c = cols[ind + l + m]
                            new_df[name] += df[cols[ind + l + m]] + df[cols[ind + l + m + 35]]

        #Next, let's bring in the data from the race-by-sex spreadsheet
        s = f'../data/NHGIS/race_by_sex/race_by_sex_{year - 2}_{year + 2}.csv'
        race_df = pd.read_csv(s,encoding='unicode_escape')

        #Drop all entries corresponding to Puerto Rico
        race_df.drop(index = race_df.index[race_df['STATE'] == "Puerto Rico"].tolist(), inplace=True)
        race_df.reset_index(inplace=True,drop=True)

        #Create fips column to properly merge with df
        race_df['fips'] = 1000* race_df['STATEA'] + race_df['COUNTYA']

        #Sort df and race_df by fips code to ensure proper merging
        race_df.sort_values(by='fips',inplace=True,ignore_index=True)
        new_df.sort_values(by='fips',inplace=True,ignore_index=True)

        #Check that the fips codes are all matched up
        mismatches = [i for i in range(len(new_df)) if new_df.loc[i,'fips'] != race_df.loc[i,'fips']]
        # print('Mismatches:', mismatches)

        #Add race-by-sex data to new_df
        sex = [['total','001'],
                ['male','002'],
                ['female','017']]
        race = [['wht',race_dict(year)[0]],
                ['blk',race_dict(year)[1]],
                ['other'] + race_dict(year)[2:]]

        for r in race:
            for s in sex:
                name = f'race_{r[0]}_{s[0]}'
                new_df[name] = 0
                for w in r[1:]:
                    labl = w + s[1]
                    new_df[name] += race_df[labl]

        #This concludes the cleaning. Now, we return the cleaned dataframe to the demographic dictionary
        demographic[year] = new_df

### Geographic Data Cleaner

In [65]:
def geo_cleaner():
    for year in years:
        # The last three years need updated column names and total area in sq-miles
        if year!=2008:
            # Rename columns for consistency
            geographic[year] = geographic[year].rename({'STATEFP':'STATE', 'COUNTYFP':'COUNTY'}, axis='columns')
            # Add columns for the total area in sqmiles      
            geographic[year]['CENSUSAREA'] = list(map( f, geographic[year]['ALAND'] + geographic[year]['AWATER'] )) 
        # Add an fips code column
        geographic[year]['fips'] = 1000 * geographic[year]['STATE'].astype(int) + geographic[year]['COUNTY'].astype(int)
        # Return cleaned dataframe to dictionary
        geographic[year] = geographic[year][['fips', 'CENSUSAREA']]
            

### Dataset Merger

In [66]:
def data_merger():
    # Merge election, demo, and geo data for each year
    for year in years:
        elec, demo, geo = fips_dropper([ election[year], demographic[year], geographic[year] ])
        data[year] = (elec.merge(demo,on='fips',how='outer')).merge(geo,on='fips',how='outer')
        
    # Drop rows that do not appear in all four years
    for i in range(4):
        data[2008+4*i] = fips_dropper([data[year] for year in data])[i]
    
    # Sort all merged data sets by FIPS codes
    for year in years:
        data[year].sort_values(by='fips',inplace=True,ignore_index=True)
        
    # Add population density by calculating population/area
    for year in years:
        data[year]['pop_density'] = data[year].pop_tot / data[year].CENSUSAREA

### Missing Value Interpolator

In [67]:
def interpolate_nans():
    for year in years:                                                                  # go through each year
        nan_dic = find_nan(data[year])                                                  # make dict with key=column names : values=list of row indices with nans
        for column in data[year].columns:                                               # go through each column
            if len(nan_dic[column])>0:
                for nan_row in nan_dic[column]:                                         # check if there's a row with a nan in that column
                    values = pd.Series([ data[y].loc[nan_row,column] for y in data ])   # if so, make a series with values for that entry across all four years
                    values = values.interpolate().bfill()             # interpolate to fill in the nan value
                    for i in range(4):                                      
                        data[2008+4*i].loc[nan_row,column] = values[i]                  # replace the values for that row,column across all four years

### Change data to percentage values

In [68]:
def to_percentages():
    # Append a Boolean row for classification
    # Dem-win = 0 , Repub-win = 1
    for year in years:
        data[year]['y'] = 0 
        for i in range(len(data[year])):
            if data[year].loc[i,'repub'] > data[year].loc[i,'dem']:
                data[year].loc[i,'y'] = 1

    # Append columns with percentage values
        new_cols = {'fips' : data[year].fips,
                'y' : data[year].y,
                'vote_diff%' : (data[year].repub - data[year].dem) / (data[year].repub + data[year].dem),
                'vote_prob' : data[year].repub / ( data[year].repub + data[year].dem),
                'dem%': data[year].dem/data[year].totalvotes,
                'repub%': data[year].repub/data[year].totalvotes,
                'third_party%':data[year].other/data[year].totalvotes,
                'turnout':data[year].totalvotes/data[year].pop_tot,
                'male%':data[year].pop_male/data[year].pop_tot,
                'female%':data[year].pop_female/data[year].pop_tot,
                'labor%':data[year].labor_total/data[year].pop_tot,
                'armed%':data[year].labor_armed/data[year].pop_tot,
                'empl%':data[year].labor_employed/data[year].pop_tot,
                'unempl%':data[year].labor_unemployed/data[year].pop_tot,
                'poverty%':data[year].income_poverty/data[year].pop_tot,
                'income_median': data[year].income_median,
                'income_percapita': data[year].income_percapita,
                'income_10%': data[year]['income_10']/data[year].pop_tot, 
                'income_10-15%':data[year]['income_10-15']/data[year].pop_tot, 
                'income_15-25%':data[year]['income_15-25']/data[year].pop_tot, 
                'income_25%':data[year].income_25/data[year].pop_tot,
                'single%': data[year].marital_single/data[year].pop_tot, 
                'married%':data[year].marital_married/data[year].pop_tot, 
                'marital_ratio': data[year].marital_single/data[year].marital_married, # switched
                'sepdiv%':data[year].marital_sepdiv/data[year].pop_tot, 
                'widow%':data[year].marital_widow/data[year].pop_tot,
                'edu_low%':data[year].edu_low/data[year].pop_tot, 
                'edu_mid%': data[year].edu_mid/data[year].pop_tot, 
                'edu_high%': data[year].edu_high/data[year].pop_tot, 
                'edulow_age18%':data[year].edu_low_age_18/data[year].pop_tot, 
                'edulow_age45%':data[year].edu_low_age_45/data[year].pop_tot, 
                'edulow_age65%':data[year].edu_low_age_65/data[year].pop_tot, 
                'edumid_age18%': data[year].edu_mid_age_18/data[year].pop_tot, 
                'edumid_age45%': data[year].edu_mid_age_45/data[year].pop_tot, 
                'edumid_age65%': data[year].edu_mid_age_65/data[year].pop_tot, 
                'eduhigh_age18%': data[year].edu_high_age_18/data[year].pop_tot, 
                'eduhigh_age45%': data[year].edu_high_age_45/data[year].pop_tot, 
                'eduhigh_age65%': data[year].edu_high_age_65/data[year].pop_tot, 
                'age_18%': data[year].edu_low_age_18/data[year].pop_tot + data[year].edu_mid_age_18/data[year].pop_tot + data[year].edu_high_age_18/data[year].pop_tot,
                'age_45%': data[year].edu_low_age_45/data[year].pop_tot + data[year].edu_mid_age_45/data[year].pop_tot + data[year].edu_high_age_45/data[year].pop_tot,
                'age_65%': data[year].edu_low_age_65/data[year].pop_tot + data[year].edu_mid_age_65/data[year].pop_tot + data[year].edu_high_age_65/data[year].pop_tot,
                'wht%': data[year].race_wht_total/data[year].pop_tot, 
                'wht_m%': data[year].race_wht_male/data[year].pop_tot, 
                'wht_f%': data[year].race_wht_female/data[year].pop_tot, 
                'blk%': data[year].race_blk_total/data[year].pop_tot, 
                'blk_m%': data[year].race_blk_male/data[year].pop_tot, 
                'blk_f%': data[year].race_blk_female/data[year].pop_tot, 
                'other%': data[year].race_other_total/data[year].pop_tot, 
                'other_m%': data[year].race_other_male/data[year].pop_tot, 
                'other_f%': data[year].race_other_female/data[year].pop_tot,
                'native%':data[year].native_yes/data[year].pop_tot,
                'hispanic%':data[year].hispanic/data[year].pop_tot,
                'pop_density':data[year].pop_tot / data[year].CENSUSAREA,
                'houses_density':data[year].houses_tot / data[year].CENSUSAREA,
                'pop_tot': data[year].pop_tot,
                'area_sqmiles': data[year].CENSUSAREA}
        data[year] = pd.DataFrame(new_cols)

## III. Data Creation Functions

In [69]:
def create_files():
    # Create csv files for each election year
    for year in years:
        data[year].to_csv(f"clean_data/data{year}.csv", index = False)

    # Create one csv file with all the years combined
    data_with_year = data.copy()

    for year in years:
        # Add a year column to each dataset
        year_col = pd.Series(  np.full( len(data[year]) , year)  )
        data_with_year[year]['year'] = year_col

    all_years = pd.concat( [data_with_year[year] for year in years] , axis=0, ignore_index=True  )

    all_years.to_csv("clean_data/all_years.csv", index = False)

In [70]:
def create_files_in_percentages():
    # Create csv files for each election year
    for year in years:
        data[year].to_csv(f"clean_data_%/data{year}.csv", index = False)

    # Create one csv file with all the years combined
    data_with_year = data.copy()

    for year in years:
        # Add a year column to each dataset
        year_col = pd.Series(  np.full( len(data[year]) , year)  )
        data_with_year[year]['year'] = year_col

    all_years = pd.concat( [data_with_year[year] for year in years] , axis=0, ignore_index=True  )

    all_years.to_csv("clean_data_%/all_years.csv", index = False)

# Data Cleaning, Merging, and Creation

In [71]:
# Clean election data
election_cleaner()
# Clean demographic data
demo_cleaner()
# Clean geographic data
geo_cleaner()

# Merge election, demographic, and geographic data by year
data_merger()
# Interpolate to recover any missing values
interpolate_nans()

# Create files
create_files()
# Change data to percentages
to_percentages()
# Create files
create_files_in_percentages()

/var/folders/26/840b8l353yv58xh3z45hs1tc0000gn/T/ipykernel_2276/3813285347.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  third_parties = election[year][ election[year]['party'] != 'DEMOCRAT'][ election[year]['party'] != 'REPUBLICAN'].index.to_list()
/var/folders/26/840b8l353yv58xh3z45hs1tc0000gn/T/ipykernel_2276/3813285347.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  third_parties = election[year][ election[year]['party'] != 'DEMOCRAT'][ election[year]['party'] != 'REPUBLICAN'].index.to_list()
/var/folders/26/840b8l353yv58xh3z45hs1tc0000gn/T/ipykernel_2276/3813285347.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  third_parties = election[year][ election[year]['party'] != 'DEMOCRAT'][ election[year]['party'] != 'REPUBLICAN'].index.to_list()
/var/folders/26/840b8l353yv58xh3z45hs1tc0000gn/T/ipykernel_2276/3813285347.py:26: UserWarning: Boolean Series key will be reindexed t